In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import numpy as np
import string
import json

In [2]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [4]:
movie_crew = pd.read_csv('Data/movie_crew.tsv', sep='\t')
name_basics = pd.read_csv('Data/names_basics.tsv', sep='\t')


In [5]:
movie_crew.head() 

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [6]:
name_basics.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0075213,tt0117057"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


In [20]:
directors = movie_crew.merge(name_basics, right_on= ['nconst'], left_on= ['directors'], how='inner')

In [21]:
directors.head()

,tconst,directors,writers,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0000001,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
1,tt0000005,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
2,tt0000006,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
3,tt0000008,nm0005690,\N,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"
4,tt0000036,nm0005690,nm0410331,nm0005690,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1496763,tt0219560,tt0308254,tt1428455"


In [29]:
Unuseful = ['birthYear', 'deathYear', 'primaryProfession', 'knownForTitles', 'writers', 'nconst', 'directors']
directors.drop(Unuseful, axis=1, inplace = True)
directors.rename(columns={'primaryName': 'Director'}, inplace=True)

In [30]:
directors.head()

,tconst,Director
0,tt0000001,William K.L. Dickson
1,tt0000005,William K.L. Dickson
2,tt0000006,William K.L. Dickson
3,tt0000008,William K.L. Dickson
4,tt0000036,William K.L. Dickson


In [ ]:
#movies_directors.to_csv("Data/movies_directors.csv")

In [31]:
title_basics = pd.read_csv('Data/title_basics.tsv', sep='\t')

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [33]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [39]:
movies_directors = title_basics.merge(directors, on='tconst', how='inner')

In [42]:
unuseful = ['titleType', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
movies_directors.drop(unuseful, axis=1, inplace = True)
movies_directors.rename(columns={'primaryTitle': 'Title'}, inplace=True)

In [43]:
movies_directors.head()

,tconst,Title,Director
0,tt0000001,Carmencita,William K.L. Dickson
1,tt0000002,Le clown et ses chiens,Émile Reynaud
2,tt0000003,Pauvre Pierrot,Émile Reynaud
3,tt0000004,Un bon bock,Émile Reynaud
4,tt0000005,Blacksmith Scene,William K.L. Dickson


In [46]:
movies_additional = pd.read_csv('Data/movies_kaggle.csv')

/var/folders/tn/v_tw25wx0cxgctjg5n0dqd0w0000gn/T/ipykernel_97329/305887195.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_additional = pd.read_csv('Data/movies_kaggle.csv')


In [47]:
movies_additional.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [50]:
movies_additional_info= movies_additional.merge(movies_directors, left_on='imdb_id', right_on='tconst', how='inner')

KeyboardInterrupt: 

In [49]:
movies_additional_info.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'tconst', 'Title', 'Director'],
      dtype='object')

In [53]:
unused = ['genres', 'homepage', 'id', 'adult', 'belongs_to_collection', 'imdb_id', 'original_language', 'original_title', 'overview', 'production_countries', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'tconst', 'Title',
       'popularity', 'poster_path']
movies_additional_info.drop(unused, axis=1, inplace = True)
movies_additional_info.rename(columns={'': ''}, inplace=True)

In [54]:
movies_additional_info.head()

,budget,production_companies,release_date,Director
0,30000000,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,John Lasseter
1,65000000,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,Joe Johnston
2,0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,Howard Deutch
3,16000000,[{'name': 'Twentieth Century Fox Film Corporat...,1995-12-22,Forest Whitaker
4,0,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995-02-10,Charles Shyer


In [55]:
#change the date of release
movies_additional_info['release_date']= movies_additional_info['release_date'].str.extract('(\d{4})')

In [58]:
movies_additional_info.rename(columns={'budget': 'Budget', 'production_companies':'Production Companies', 'release_date':'ReleaseYear'})

,Budget,Production Companies,ReleaseYear,Director
0,30000000,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995,John Lasseter
1,65000000,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995,Joe Johnston
2,0,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995,Howard Deutch
3,16000000,[{'name': 'Twentieth Century Fox Film Corporat...,1995,Forest Whitaker
4,0,"[{'name': 'Sandollar Productions', 'id': 5842}...",1995,Charles Shyer
...,...,...,...,...
41288,0,[],NaN,Hamid Nematollah
41289,0,"[{'name': 'Sine Olivia', 'id': 19653}]",2011,Lav Diaz
41290,0,"[{'name': 'American World Pictures', 'id': 6165}]",2003,Mark L. Lester
41291,0,"[{'name': 'Yermoliev', 'id': 88753}]",1917,Yakov Protazanov


In [60]:
movies_additional_info.to_csv("Data/movies_additional_info.csv")